In [ ]:
!pip install tensorflow

In [ ]:
from google.colab import drive
import os
import pandas as pd
filename = 'data_password_only.csv'
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/DSC Projects/') #Change to your final project directory
print(os.listdir())
df = pd.read_csv(filename, header=None, dtype=str)
df.columns = ['password']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['NLP.ipynb', 'output_1.xlsx', 'output_1.xlsx_password_analysis.xlsx', 'password_statistics.csv', 'output_5.xlsx', 'output_3.xlsx', 'output_7.xlsx', 'profanity_en.csv', 'output_6.xlsx', 'output_4.xlsx', 'output_2.xlsx_password_analysis.xlsx', 'output_2.xlsx', 'data.csv', 'data_analysis.ipynb', 'Final Project Proposal.gdoc', 'data.gsheet', 'data_password_only.csv', 'passwords.txt', 'cached_lm_GPT2Tokenizer_32_passwords.txt.lock', 'cached_lm_GPT2Tokenizer_32_passwords.txt', 'password_generator', 'wandb', 'Recurrent Neural Network (RNN).ipynb']


In [ ]:
df = df.head(100000)
df.shape

(100000, 1)

In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical

all_text = ''.join(df['password'].astype(str).tolist())
chars = sorted(set(all_text))
char_to_index = {c: i for i, c in enumerate(chars)}
index_to_char = {i: c for i, c in enumerate(chars)}

In [ ]:
sequence_length = 10

sequences = []
next_chars = []
for i in range(0, len(all_text) - sequence_length):
    sequences.append(all_text[i: i + sequence_length])
    next_chars.append(all_text[i + sequence_length])

# Vectorize sequences and next characters
X = np.zeros((len(sequences), sequence_length, len(chars)), dtype=np.bool_)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool_)

for i, seq in enumerate(sequences):
    for t, char in enumerate(seq):
        X[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_chars[i]]] = 1


X.shape, y.shape

((998353, 10, 129), (998353, 129))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(128, input_shape=(sequence_length, len(chars))),
    Dense(len(chars), activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.fit(X, y, epochs=3, batch_size=128)


Epoch 1/3
7800/7800 ━━━━━━━━━━━━━━━━━━━━ 279s 36ms/step - loss: 3.4044
Epoch 2/3
7800/7800 ━━━━━━━━━━━━━━━━━━━━ 326s 36ms/step - loss: 3.2286
Epoch 3/3
7800/7800 ━━━━━━━━━━━━━━━━━━━━ 279s 36ms/step - loss: 3.1907


In [ ]:
def generate_text(model, seed_text, length=50, temperature=1.0):
    """
    Generate text using the trained model.
    - model: Trained LSTM model.
    - seed_text: Initial string to start generating text.
    - length: Number of characters to generate.
    - temperature: Controls randomness in predictions. Lower values make the text more predictable.
    """
    generated_text = seed_text
    for _ in range(length):
        # Vectorize the seed_text
        sampled = np.zeros((1, len(seed_text), len(chars)))
        for i, char in enumerate(seed_text):
            sampled[0, i, char_to_index[char]] = 1

        # Predict the next character
        predictions = model.predict(sampled, verbose=0)[0]
        predictions = np.asarray(predictions).astype('float64')

        # Apply temperature to predictions to control randomness
        predictions = np.log(predictions + 1e-8) / temperature
        exp_preds = np.exp(predictions)
        predictions = exp_preds / np.sum(exp_preds)

        # Sample the next character based on adjusted probabilities
        next_index = np.random.choice(range(len(chars)), p=predictions)
        next_char = index_to_char[next_index]

        # Append the next character to generated_text and update seed_text
        generated_text += next_char
        seed_text = seed_text[1:] + next_char  # Move window forward

    return generated_text


In [ ]:
# Example seed text (choose a starting sequence)
seed_text = "119*02"  # or select a common password pattern in your dataset

# Generate new password-like text
generated_password = generate_text(model, seed_text, length=10, temperature=0.5)
print("Generated password:", generated_password)


Generated password: 119*0219699001ma


In [ ]:
model_save_path = 'models/recurrentNeuralNetwork.h5'

# Save the model
model.save(model_save_path)